### LAB02 - AI Plugins in Semantic Kernel

In this lab, we will learn how to use Semantic Kernel to build native and semantic functions and group them together as AI Plugins. We'll work on the following steps:

1) What are AI Plugins? 
2) Create and add a Semantic Function to a Plugin.
3) Create and add a Native Function to a Plugin.
4) Puting it all together.

#### What are AI Plugins?
As the Copilot concept evolves, it's important to think on ways to extend their capabilities, allowing them to retrieve external information, interact with external services and execute actions on the behavior of the user on a safer way, following Responsible AI principles.

The concept of **Plugins** is an answer to this, as they can act as a "bridge" between Copilots / AI Apps and the digital world. 
![Plugins Overview](images/plugins_overview.png)

With Plugins, it's possible to expand Copilots capabilities and promote interoperability across the industry as Semantic Kernel adopts an open standard for plugins, the [OpenAPI Specification](https://swagger.io/specification/).

Let's get started with the creation of Functions, which will be the base of our AI Plugins. In Semantic Kernel we can think of Plugins as a collection of functions. Let's see in the following sections how we can create functions and group them together as a Plugin. 

#### Create and add a Semantic Function to a Plugin

In the first lesson, we have created and used an [**inline**](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=python) Semantic Function to categorize a call.However this approach is fine for development and testing, we need a more robust way to ensure we can reuse the functions across projects and rapidly maintain / change the prompts and function parameters as required.

Therefore, we'll now explore how we can reuse such functions. Semantic Kernel allows us to import files to define functions. We must follow a standard folder structure when creating the files. Here's an example:

```
📁 plugins
│
└─── 📂 plugin_name_A
     |
     └─── 📂 function_name_A 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     └─── 📂 function_name_B 
            |
            └───📄 skprompt.txt
            └───📄 config.json
    📂 plugin_name_B
     |
     └─── 📂 function_name_C 
            |
            └───📄 skprompt.txt
            └───📄 config.json
```

With this structure in mind, we already start organizing our plugins and grouping together functions that belong to a plugin. For each Function (represented by a Folder) we must have two files:
* **skprompt.txt**: it's the file that will contain the prompt of a Semantic Function. Therefore, we can easily maintain our functions by reviewing the prompts;
* **config.json**: it's a configuration file that will describe the function in natural language, define which parameters the function requires, and the overall configuration the LLM should use run that function, such as temperature and maximum number of tokens. 

With this structure in mind, we already defined a folder structure to contain the funcions for a Call Center solution plugin. In this lesson, we'll recreate the function used before to fit into this structure.

Copy and paste the prompt below in the skprompt.txt file inside the Semantic Function "categorize" under "callcenter" plugin:

```txt
You help a Telecom company to classify problems reported by their subscribers. Your role is to provide accurate classification based on problems description and a list of categories that the Telecom company uses.
Classify the problem in one of the provided categories. Only write the output category with no extra text.
Only write one category per problem description.

Examples: 
Problem: My 5G is not working well when I'm in my car.
Categories: Fixed Internet/Wifi, Mobile Internet.
Output Category: Mobile Internet

Problem: My TV is not streaming well from Youtube. It seems that the wifi in my bedroom is weak.
Categories: Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: It's impossible to work today, my internet here in my home is so slow!
Categories: Landiline, Mobile Internet, TV, Fixed Internet/Wifi.
Output Category: Fixed Internet/Wifi

Problem: {{$problem}}
Categories: {{$categories}}
Output Category: 
```

Next, modify the config.json file to ensure it looks like these:
```json
{
     "schema": 1,
     "type": "completion",
     "description": "Categorize a problem reported in a call to the Telecom company based on provided categories.",
     "completion": {
          "max_tokens": 10,
          "temperature": 0.5,
          "top_p": 0.0,
          "presence_penalty": 0.0,
          "frequency_penalty": 0.0
     },
     "input": {
          "parameters": [
               {
                    "name": "problem",
                    "description": "The problem reported by the user that needs to be classified.",
                    "defaultValue": ""
               },
               {
                    "name": "categories",
                    "description": "A set of categories used by the Telecom company to classify the problems.",
                    "defaultValue": ""
               }
          ]
     }
}
```

At this moment, your folder structure should have at least the structure below.
```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
```

Let's define a variable to represent our Plugins folder.

In [84]:
#Define where the plugins are stored
plugins_directory = "../plugins"

To manipulate functions, we need to ensure we have all our pre-requirements set, as we did in the previous lesson. 

In [85]:
import semantic_kernel as sk

#Importing the Azure Text Completion service connector
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion

# Initialize the kernel
kernel = sk.Kernel()

#Read the model, API key and endpoint from the .env file
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env() 

#Since our function is a Text Completion, we need to add it to the kernel
kernel.add_text_completion_service("Text Completion - Text-DaVinci-003", 
                                   AzureTextCompletion(deployment, endpoint, api_key))

Now we can refer to our Plugin and import it to be able to use its functions.

In [86]:
# Importing the callcenter Plugin from the plugins directory, so we can start using its functions 
callcenter_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "CallCenterPlugin"
)

In both **config.json** and **skprompt.txt** files we have defined input parameters that "**Categorize**" function requires. In Semantic Kernel we need to use **ContextVariables** to bind values with such parameters. 

After defining the input variables, we can finally call our semantic function available in our Plugin to perform the expected operation. Please note in the code that we can have multiple functions in a Plugin, so in this case we had to explicitly mention which semantic function we would like to use.

In [87]:
#Defining the input variables for the function
variables = sk.ContextVariables()
variables["problem"] = "My 3G is terrible slow ."
variables["categories"] = "Landline, Mobile Internet, TV, Fixed Internet/Wifi"

#Call the plugin with input variables
result = await kernel.run_async(
    callcenter_plugin["categorize"],
    input_vars = variables
    )

print(result)

 Mobile Internet


### Create and add a Native Function to a Plugin

When we think about the concept of AI Apps it's important to remember that an application will certainly require capabilities that will not be provided solely by Large Language Models. For example, performing calculations, retrieving data from a database and even executing API calls are some of the expected capabilities an application generally has. 

Therefore, we still need to run native code and be able to combine it with AI capabilities to go beyond and create transformative applications powered by modern AI!

In semantic Kernel we can add native code in different languages, like C# and Python, to be part of our Plugins. We should use the same folder structure from before, however, instead of prompts and configurations, we will have native code, like .py (Python) or .cs (C#) files.

For example, let's assume that we want to add a new function to "CallCenter" Plugin to transcribe audio files using. Let's see how our folder structure would look like:

```
📁 plugins
│
└─── 📂 CallCenterPlugin
     |
     └─── 📂 categorize 
     |      |
     |      └───📄 skprompt.txt
     |      └───📄 config.json
     |
     └─── 📂 transcribe
            |   
            └───📄 transcribe.py
```

Let's assume we already have a base code in Python that is capable of transcribing an audio file to text, as shown below:

```python
import azure.cognitiveservices.speech as speechsdk

# Define the transcribe_audio function
def transcribe_audio(audio_file, language, subscription_key, region):
    # Set up the speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    # Define the language of the audio file
    speech_config.speech_recognition_language = language

    # Open the audio file
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Define global variables
    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]

    # Define the stop callback function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    # Define the recognize callback function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    speech_recognizer.session_stopped.connect(stop_cb)
 
    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    # Stop continuous speech recognition
    speech_recognizer.stop_continuous_recognition()
    
    # Convert to string and return the transcription
    return recognized_text_list
```



 We need to do some small changes in the code to ensure it can become a native function and can be addedd to our Plugin in Semantic Kernel. In summary:
- **Transform it in a class:** all native functions must be defined as public methods that belong to a class. This class will represent our Plugin.
- **Add the decorator @sk_function:** the kernel will use this decorator to be able to register this function as the plugin is loaded. The decorator will also describe the function and its input parameter (if only one). Such information can be later used by Planners to orchestrate Plugins.
- **Add the decorator sk_function_context_parameter:** if your function has multiple input parameters you need to use this decorator to describe the input parameters with their names and description, so the kernel understand how to call the function.
-**Import the required libraries:** you need to import the skill_definition module. 

Let's see our modified code. Ensure the code from the cell below is also present in the file **"transcribe.py"** in the plugins directory.

In [29]:
# Import the time module
import time

# Define the transcribe_audio function
def transcribe_audio(audio_file, language, subscription_key, region):
    # Set up the speech configuration
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    # Define the language of the audio file
    speech_config.speech_recognition_language = language

    # Open the audio file
    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)

    # Create a speech recognizer
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Define global variables
    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]

    # Define the stop callback function
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    # Define the recognize callback function
    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    speech_recognizer.session_stopped.connect(stop_cb)
 
    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    # Stop continuous speech recognition
    speech_recognizer.stop_continuous_recognition()
    
    # Convert to string and return the transcription
    return recognized_text_list

# Usage example
audio_file = "./audio/english_billing_process_sample.wav"
language = "en-US"
subscription_key = "8198d6d7eb8341ba90022f1224ee8967"
region = "brazilsouth"

# Call the transcribe_audio function
transcription = transcribe_audio(audio_file, language, subscription_key, region)
print(transcription)

STT SESSION STARTED: SessionEventArgs(session_id=bed631eb26aa4822b1533c6bdd89162b)
STT SESSION STOPPED SessionEventArgs(session_id=bed631eb26aa4822b1533c6bdd89162b)
CLOSING on SessionEventArgs(session_id=bed631eb26aa4822b1533c6bdd89162b)
Thank you for calling us.Who am I speaking with?Hello, my name is Peter Smith.I have a small business and have some questions about billing processing. Good morning, Peter.Before we begin, may I ask some questions to better answer your questions today, yes.In case we get disconnected, can you share your phone number and e-mail address so we can get in touch and share additional information?Yes, my phone number is 02199991234 and my e-mail address is peter.smith@contoso.com. May I ask where your business is located so I can answer your questions based on your location? Yes, my business is located in Dallas state of Texas.Thank you. How can I help you today? I am relatively new to this process and currently do everything manually.Do you have any advice o

In [122]:
from plugins.CallCenterPlugin.Transcribe import Transcribe

# Importing the callcenter Plugin from the plugins directory, so we can start using its functions 
#callcenter_plugin = kernel.import_native_skill_from_directory(plugins_directory, "CallCenterPlugin")
#import_native_skill_from_directory(
    #plugins_directory, "CallCenterPlugin"
#)

aispeech_subscription_key = "REPLACE"
aispeech_region = "brazilsouth"

#Defining the input variables for the transcription funcion
variables = sk.ContextVariables()
variables["audio"] = "./audio/english_billing_process_sample.wav"
variables["language"] = "en-US"
variables["subscription_key"] = aispeech_subscription_key
variables["region"] = aispeech_region

callcenter_plugin = kernel.import_skill(Transcribe(), skill_name="CallCenterPlugin")

#Call the plugin with input variables
result = await kernel.run_async(
    callcenter_plugin["audio1"],
    input_vars = variables
    )

print(result)

KernelException: (<ErrorCodes.FunctionTypeNotSupported: 3>, 'Invalid function type detected, unable to infer DelegateType. Function: (language, subscription_key, region)', None)